In [30]:
# -------------------------------
# 1. Imports
# -------------------------------
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains import RetrievalQA

In [31]:
# -------------------------------
# 2. Initialize Ollama LLM
# -------------------------------
llm = Ollama(
    model="llama3",
    base_url="http://localhost:11434",
    temperature=0
)

In [32]:
# -------------------------------
# 3. Initialize Embeddings
# -------------------------------
embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
    base_url="http://localhost:11434"
)

In [33]:
# -------------------------------
# 4. Create Vector Store
# -------------------------------
vector_store = Chroma(
    collection_name="my_rag",
    embedding_function=embeddings
)

In [34]:
# -------------------------------
# 5. Add Documents
# -------------------------------
docs = [
    Document(page_content="Ollama runs locally."),
    Document(page_content="Gemini requires API keys."),
    Document(page_content="The average transaction amount is 2,500 INR.")
]

vector_store.add_documents(docs)


['450cbd54-7660-4182-94f1-991252a09be7',
 '577ba72d-c99a-4dbd-975e-0f48850687fb',
 '7e679435-0402-4d58-ab88-6e81866c2bd8']

In [35]:
# -------------------------------
# 6. Create Retriever
# -------------------------------
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [36]:
# -------------------------------
# 7. Prompt Template
# -------------------------------
policy_prompt = """
You are a helpful assistant.
Use ONLY the provided context to answer the question.
If the answer is not in the context, say "I don't know".
"""

prompt = PromptTemplate(
    template=policy_prompt + "\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:",
    input_variables=["context", "question"]
)

In [37]:

# -------------------------------
# 8. RetrievalQA Chain
# -------------------------------
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

In [38]:
# -------------------------------
# 9. Queries
# -------------------------------
print(qa_chain.invoke("Summarize the average transaction amount from the stored data."))
print(qa_chain.invoke("What is the average transaction amount?"))

{'query': 'Summarize the average transaction amount from the stored data.', 'result': 'According to the provided context, the average transaction amount is 2,500 INR.'}
{'query': 'What is the average transaction amount?', 'result': 'According to the provided context, the answer is:\n\n2,500 INR'}
